In [1]:
!pip install -q networkx codecarbon pandas matplotlib scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.1/278.1 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.7/258.7 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 kB 2.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.46.0 requires httpx<1.0.0,>=0.28.1, but you have httpx 0.27.2 which is incompatible.
firebase-admin 6.9.0 requires httpx[http2]==0.28.1, but you have httpx 0.27.2 which is incompatible.


In [2]:
import os
import random
import time
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from scipy import stats
from codecarbon import EmissionsTracker

In [3]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

In [4]:
class MinHeap:
    def __init__(self, array):
        self.vertexMap = {idx: idx for idx in range(len(array))}
        self.heap = self.buildHeap(array)

    def isEmpty(self): return len(self.heap) == 0

    def buildHeap(self, array):
        firstParentIdx = (len(array) - 2) // 2
        for currentIdx in reversed(range(firstParentIdx + 1)):
            self.siftDown(currentIdx, len(array) - 1, array)
        return array

    def siftDown(self, currentIdx, endIdx, heap):
        child_one_idx = currentIdx * 2 + 1
        while child_one_idx <= endIdx:
            child_two_idx = currentIdx * 2 + 2 if currentIdx * 2 + 2 <= endIdx else -1
            if child_two_idx != -1 and heap[child_two_idx][1] < heap[child_one_idx][1]:
                idx_to_swap = child_two_idx
            else:
                idx_to_swap = child_one_idx
            if heap[idx_to_swap][1] < heap[currentIdx][1]:
                self.swap(currentIdx, idx_to_swap, heap)
                currentIdx = idx_to_swap
                child_one_idx = currentIdx * 2 + 1
            else:
                return

    def siftUp(self, currentIdx, heap):
        parentIdx = (currentIdx - 1) // 2
        while currentIdx > 0 and heap[currentIdx][1] < heap[parentIdx][1]:
            self.swap(currentIdx, parentIdx, heap)
            currentIdx = parentIdx
            parentIdx = (currentIdx - 1) // 2

    def remove(self):
        if self.isEmpty(): return None
        self.swap(0, len(self.heap) - 1, self.heap)
        vertex, distance = self.heap.pop()
        self.vertexMap.pop(vertex)
        self.siftDown(0, len(self.heap) - 1, self.heap)
        return vertex, distance

    def swap(self, i, j, heap):
        self.vertexMap[heap[i][0]] = j
        self.vertexMap[heap[j][0]] = i
        heap[i], heap[j] = heap[j], heap[i]

    def update(self, vertex, value):
        self.heap[self.vertexMap[vertex]] = (vertex, value)
        self.siftUp(self.vertexMap[vertex], self.heap)

In [5]:
# Dijkstra clássico (O(V²+E)) – versão da aula
def dijkstra_classico(start, edges):
    n = len(edges)
    INF = float('inf')
    dist = [INF] * n
    visited = [False] * n
    dist[start] = 0

    for _ in range(n):
        # vértice não visitado com menor distância (busca linear)
        v = -1
        best = INF
        for i in range(n):
            if not visited[i] and dist[i] < best:
                best = dist[i]
                v = i
        if v == -1 or best == INF:   # restante inacessível
            break
        visited[v] = True
        for to, w in edges[v]:
            if not visited[to]:
                nd = best + w
                if nd < dist[to]:
                    dist[to] = nd
    return [-1 if x == INF else x for x in dist]

In [6]:
# Dijkstra com Min‑Heap (O((V+E)logV)) – versão da aula
def dijkstra_min_heap(start, edges):
    n = len(edges)
    INF = float('inf')
    dist = [INF] * n
    dist[start] = 0

    heap = MinHeap([(i, INF) for i in range(n)])
    heap.update(start, 0)

    while not heap.isEmpty():
        v, d = heap.remove()
        if d != dist[v]:          # ignore outdated entry
            continue
        for to, w in edges[v]:
            nd = d + w
            if nd < dist[to]:
                dist[to] = nd
                heap.update(to, nd)
    return [-1 if x == INF else x for x in dist]

In [7]:
def medir_tempo(func, *args):
    t0 = time.time()
    _ = func(*args)
    return time.time() - t0

def medir_com_codecarbon(func, *args):
    tracker = EmissionsTracker()
    tracker.start()
    elapsed = medir_tempo(func, *args)
    co2 = tracker.stop()
    return elapsed, co2

In [8]:
def gerar_grafo(n, p=0.01, peso_min=1, peso_max=10):
    G = nx.gnp_random_graph(n, p, seed=SEED, directed=True)
    # componente gigante
    if not nx.is_strongly_connected(G):
        comp = nx.strongly_connected_components(G)
        giant = max(comp, key=len)
        G = G.subgraph(giant).copy()
        # adiciona arestas aleatórias até ficar fortemente conectado
        while not nx.is_strongly_connected(G):
            u, v = random.sample(list(G.nodes), 2)
            if not G.has_edge(u, v):
                w = random.randint(peso_min, peso_max)
                G.add_edge(u, v, weight=w)
    for u, v in G.edges():
        G[u][v]['weight'] = random.randint(peso_min, peso_max)
    return G

In [ ]:
tamanhos = [100, 500, 1_000, 5_000, 10_000, 50_000]
repeticoes_por_tamanho = 15          # 15‑20 recomendado;
sources_por_rep = 5                 # número de fontes por repetição

resultados = []   # lista de dicts

for n in tamanhos:
    p = max(0.005, 1e-5 * n) if n <= 10_000 else 0.0002
    print(f'Gerando grafo com {n} nós (p={p:.5f})...')
    G = gerar_grafo(n, p=p)

    # ---- Conversão para a estrutura esperada pelos algoritmos ----
    adj = [[] for _ in range(n)]
    for u, nbrs in G.adjacency():
        for v, data in nbrs.items():
            adj[u].append([v, data['weight']])

    for rep in range(repeticoes_por_tamanho):
        max_src = min(sources_por_rep, len(G))
        sources = random.sample(list(G.nodes), max_src)

        for s in sources:
            # ----- Dijkstra clássico -----
            t_c, co2_c = medir_com_codecarbon(dijkstra_classico, s, adj)
            resultados.append({
                'n': n,
                'algoritmo': 'classico',
                'tempo': t_c,
                'co2': co2_c
            })

            # ----- Dijkstra com heap -----
            t_h, co2_h = medir_com_codecarbon(dijkstra_min_heap, s, adj)
            resultados.append({
                'n': n,
                'algoritmo': 'heap',
                'tempo': t_h,
                'co2': co2_h
            })

            # ----- NetworkX (referência) -----
            t_n, co2_n = medir_com_codecarbon(
                lambda start: nx.single_source_dijkstra_path_length(G, start, weight='weight'),
                s
            )
            resultados.append({
                'n': n,
                'algoritmo': 'networkx',
                'tempo': t_n,
                'co2': co2_n
            })

[codecarbon WARNING @ 22:50:10] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 22:50:10] [setup] RAM Tracking...
[codecarbon INFO @ 22:50:10] [setup] CPU Tracking...


Gerando grafo com 100 nós (p=0.00500)...


[codecarbon WARNING @ 22:50:11] We saw that you have a Intel(R) Xeon(R) CPU @ 2.20GHz but we don't know it. Please contact us.
[codecarbon WARNING @ 22:50:11] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 22:50:11] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon WARNING @ 22:50:11] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 22:50:11] [setup] GPU Tracking...
[codecarbon INFO @ 22:50:11] No GPU found.
[codecarbon INFO @ 22:50:11] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: global constant
                GPU Tracking Method: Unspecified
            
[codecarbon INFO @ 22:50:11] >>> Tracker's metadata:
[codecarbon INFO @ 22:50:11]   Platform system: Linux-6

Gerando grafo com 500 nós (p=0.00500)...


Streaming output truncated to the last 5000 lines.
[codecarbon INFO @ 22:52:41] No GPU found.
[codecarbon INFO @ 22:52:41] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: global constant
                GPU Tracking Method: Unspecified
            
[codecarbon INFO @ 22:52:41] >>> Tracker's metadata:
[codecarbon INFO @ 22:52:41]   Platform system: Linux-6.6.105+-x86_64-with-glibc2.35
[codecarbon INFO @ 22:52:41]   Python version: 3.12.12
[codecarbon INFO @ 22:52:41]   CodeCarbon version: 3.0.7
[codecarbon INFO @ 22:52:41]   Available RAM : 12.671 GB
[codecarbon INFO @ 22:52:41]   CPU count: 2 thread(s) in 1 physical CPU(s)
[codecarbon INFO @ 22:52:41]   CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon INFO @ 22:52:41]   GPU count: None
[codecarbon INFO @ 22:52:41]   GPU model: None
[codecarbon INFO @ 22:52:42] Emissions data (if any) will be saved to file /content/emissions.csv
[codec

...

Streaming output truncated to the last 5000 lines.
[codecarbon INFO @ 23:04:42] No GPU found.
[codecarbon INFO @ 23:04:42] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: global constant
                GPU Tracking Method: Unspecified
            
[codecarbon INFO @ 23:04:42] >>> Tracker's metadata:
[codecarbon INFO @ 23:04:42]   Platform system: Linux-6.6.105+-x86_64-with-glibc2.35
[codecarbon INFO @ 23:04:42]   Python version: 3.12.12
[codecarbon INFO @ 23:04:42]   CodeCarbon version: 3.0.7
[codecarbon INFO @ 23:04:42]   Available RAM : 12.671 GB
[codecarbon INFO @ 23:04:42]   CPU count: 2 thread(s) in 1 physical CPU(s)
[codecarbon INFO @ 23:04:42]   CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon INFO @ 23:04:42]   GPU count: None
[codecarbon INFO @ 23:04:42]   GPU model: None
[codecarbon INFO @ 23:04:42] Emissions data (if any) will be saved to file /content/emissions.csv
[codec

Gerando grafo com 10000 nós (p=0.10000)...


Streaming output truncated to the last 5000 lines.
[codecarbon INFO @ 23:17:13] No GPU found.
[codecarbon INFO @ 23:17:13] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: global constant
                GPU Tracking Method: Unspecified
            
[codecarbon INFO @ 23:17:13] >>> Tracker's metadata:
[codecarbon INFO @ 23:17:13]   Platform system: Linux-6.6.105+-x86_64-with-glibc2.35
[codecarbon INFO @ 23:17:13]   Python version: 3.12.12
[codecarbon INFO @ 23:17:13]   CodeCarbon version: 3.0.7
[codecarbon INFO @ 23:17:13]   Available RAM : 12.671 GB
[codecarbon INFO @ 23:17:13]   CPU count: 2 thread(s) in 1 physical CPU(s)
[codecarbon INFO @ 23:17:13]   CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon INFO @ 23:17:13]   GPU count: None
[codecarbon INFO @ 23:17:13]   GPU model: None
[codecarbon INFO @ 23:17:13] Emissions data (if any) will be saved to file /content/emissions.csv
[codec

Gerando grafo com 50000 nós (p=0.00020)...


[codecarbon WARNING @ 23:37:06] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 23:37:06] [setup] RAM Tracking...
[codecarbon INFO @ 23:37:06] [setup] CPU Tracking...
[codecarbon WARNING @ 23:37:07] We saw that you have a Intel(R) Xeon(R) CPU @ 2.20GHz but we don't know it. Please contact us.
[codecarbon WARNING @ 23:37:07] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 23:37:07] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.20GHz
[codecarbon WARNING @ 23:37:07] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 23:37:07] [setup] GPU Tracking...
[codecarbon INFO @ 23:37:07] No GPU found.
[codecarbon INFO @ 23:37:07] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
               

In [ ]:
df = pd.DataFrame(resultados)

def ic(series, alpha=0.95):
    mean = series.mean()
    sem = stats.sem(series)
    ci = sem * stats.t.ppf((1 + alpha) / 2., len(series) - 1)
    return mean, ci

agg = df.groupby(['n', 'algoritmo']).agg(
    tempo_medio=('tempo', 'mean'),
    tempo_std=('tempo', 'std'),
    co2_medio=('co2', 'mean'),
    co2_std=('co2', 'std')
).reset_index()

In [ ]:
plt.figure(figsize=(8, 4))
for alg in ['classico', 'heap', 'networkx']:
    sub = agg[agg.algoritmo == alg]
    plt.errorbar(sub['n'], sub['tempo_medio'],
                 yerr=sub['tempo_std'], label=alg, marker='o')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Número de nós')
plt.ylabel('Tempo (s) – média ± desvio')
plt.legend()
plt.tight_layout()
plt.savefig('tempo.png')
plt.show()

plt.figure(figsize=(8, 4))
for alg in ['classico', 'heap', 'networkx']:
    sub = agg[agg.algoritmo == alg]
    plt.errorbar(sub['n'], sub['co2_medio'],
                 yerr=sub['co2_std'], label=alg, marker='o')
plt.xscale('log')
plt.yscale('log')
plt.xlabel('Número de nós')
plt.ylabel('CO₂ (kg) – média ± desvio')
plt.legend()
plt.tight_layout()
plt.savefig('co2.png')
plt.show()

In [ ]:
# opcional
df.to_csv('resultados_raw.csv', index=False)
agg.to_csv('resultados_agg.csv', index=False)

print('Experimento concluído – arquivos "tempo.png" e "co2.png" gerados.')